In [17]:
using CSV, DataFrames, Random, Gurobi, JuMP, Statistics, ScikitLearn, MLDataUtils, MLJ

In [18]:
data = CSV.read("Input/known_behaviors2.csv", DataFrame)

Row,customer_id,age,age_youngest_child,debt_equity,gender,bad_payment,gold_card,pension_plan,household_debt_to_equity_ratio,income,members_in_household,months_current_account,months_customer,call_center_contacts,loan_accounts,number_products,number_transactions,non_worker_percentage,white_collar_percentage,rfm_score,Mortgage,Pension,Savings
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64
1,15,45,12,45,0,0,0,0,65,13453,2,33,36,0,4,2,1,14,19,7.602,0,0,0
2,16,43,12,43,0,0,0,0,65,13453,2,34,36,0,0,3,2,14,19,10.143,0,0,0
3,30,23,0,23,0,0,0,0,65,13453,2,30,36,0,1,0,0,14,19,0.0,0,0,0
4,42,35,8,35,1,0,0,0,65,13453,2,22,24,0,1,0,0,14,19,0.0,0,1,0
5,52,43,12,43,1,0,0,0,47,14124,3,40,48,3,1,0,0,16,35,0.0,0,1,0
6,57,51,19,51,1,0,0,0,50,14838,2,33,36,3,0,0,0,9,39,0.0,0,0,0
7,74,31,0,31,1,0,0,0,50,14838,3,0,12,3,3,0,0,9,39,0.0,1,0,0
8,74,31,0,31,1,0,0,0,50,14838,3,0,12,3,3,0,0,9,39,0.0,1,0,0
9,89,46,11,46,1,0,0,0,32,14783,2,6,12,6,3,2,1,25,42,7.799,0,1,0


In [19]:
data[:, :concatenated] = string.(data[:, :Mortgage]) .* string.(data[:, :Pension]) .* string.(data[:, :Savings]);

In [20]:
data

Row,customer_id,age,age_youngest_child,debt_equity,gender,bad_payment,gold_card,pension_plan,household_debt_to_equity_ratio,income,members_in_household,months_current_account,months_customer,call_center_contacts,loan_accounts,number_products,number_transactions,non_worker_percentage,white_collar_percentage,rfm_score,Mortgage,Pension,Savings,concatenated
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,String
1,15,45,12,45,0,0,0,0,65,13453,2,33,36,0,4,2,1,14,19,7.602,0,0,0,000
2,16,43,12,43,0,0,0,0,65,13453,2,34,36,0,0,3,2,14,19,10.143,0,0,0,000
3,30,23,0,23,0,0,0,0,65,13453,2,30,36,0,1,0,0,14,19,0.0,0,0,0,000
4,42,35,8,35,1,0,0,0,65,13453,2,22,24,0,1,0,0,14,19,0.0,0,1,0,010
5,52,43,12,43,1,0,0,0,47,14124,3,40,48,3,1,0,0,16,35,0.0,0,1,0,010
6,57,51,19,51,1,0,0,0,50,14838,2,33,36,3,0,0,0,9,39,0.0,0,0,0,000
7,74,31,0,31,1,0,0,0,50,14838,3,0,12,3,3,0,0,9,39,0.0,1,0,0,100
8,74,31,0,31,1,0,0,0,50,14838,3,0,12,3,3,0,0,9,39,0.0,1,0,0,100
9,89,46,11,46,1,0,0,0,32,14783,2,6,12,6,3,2,1,25,42,7.799,0,1,0,010


In [22]:
stratify_columns = [:concatenated]
strata = data[:, stratify_columns]
ratio = 0.8
train, test = splitobs(1:size(strata, 1), ratio)
X_train, X_test = data[train, :], data[test, :];

In [23]:
target_columns = [:Mortgage, :Pension, :Savings]
y_train = X_train[:, target_columns]
y_test = X_test[:, target_columns]
target_columns = [:Mortgage, :Pension, :Savings, :concatenated]
X_train = X_train[:, Not(target_columns)]
X_test = X_test[:, Not(target_columns)];

In [24]:
# Split the data into training and testing sets
X_train = X_train[:, Not(:customer_id)]
X_test = X_test[:, Not(:customer_id)];

In [25]:
@sk_import preprocessing: StandardScaler

┌ Info: mkl not found, proceeding to installing non-mkl versions of sci-kit learn via Conda
└ @ ScikitLearn.Skcore /Users/zekiyan/.julia/packages/ScikitLearn/sqLdT/src/Skcore.jl:191
┌ Info: Running `conda install -y -c conda-forge 'scikit-learn>=1.2,<1.3'` in root environment
└ @ Conda /Users/zekiyan/.julia/packages/Conda/2lg2O/src/Conda.jl:127


Error while loading conda entry point: conda-libmamba-solver (dlopen(/Users/zekiyan/.julia/conda/3/aarch64/lib/python3.10/site-packages/libmambapy/bindings.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/libarchive.19.dylib
  Referenced from: <494304CB-2F24-3FF8-B4AE-F4E6B16BDC26> /Users/zekiyan/.julia/conda/3/aarch64/lib/libmamba.2.0.0.dylib
  Reason: tried: '/Users/zekiyan/.julia/conda/3/aarch64/lib/libarchive.19.dylib' (no such file), '/Users/zekiyan/.julia/conda/3/aarch64/lib/python3.10/site-packages/libmambapy/../../../libarchive.19.dylib' (no such file), '/Users/zekiyan/.julia/conda/3/aarch64/lib/python3.10/site-packages/libmambapy/../../../libarchive.19.dylib' (no such file), '/Users/zekiyan/.julia/conda/3/aarch64/bin/../lib/libarchive.19.dylib' (no such file), '/Users/zekiyan/.julia/conda/3/aarch64/bin/../lib/libarchive.19.dylib' (no such file), '/usr/local/lib/libarchive.19.dylib' (no such file), '/usr/lib/libarchive.19.dylib' (no such file, not in dyld cache))


done
Solving environment: ...working... 

done



# All requested packages already installed.



PyObject <class 'sklearn.preprocessing._data.StandardScaler'>

In [27]:
scaler = StandardScaler()

columns = names(X_train)

X_train_scaled = X_train
X_test_scaled = X_test

X_train_to_scale = Matrix(X_train[:, columns])
X_test_to_scale = Matrix(X_test[:, columns])

scaler = StandardScaler()

X_train_scaled_features = fit_transform!(scaler, X_train_to_scale)
X_test_scaled_features = transform!(scaler, X_test_to_scale)

# Replace the original values in the DataFrame with the scaled values
X_train_scaled[:, columns] .= X_train_scaled_features
X_test_scaled[:, columns] .= X_test_scaled_features



MethodError: MethodError: no method matching transform!(::PyCall.PyObject, ::Matrix{Float64})

Closest candidates are:
  transform!(!Matched::SubDataFrame, ::Any...; renamecols, threads)
   @ DataFrames /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/DataFrames/LteEl/src/abstractdataframe/selection.jl:985
  transform!(!Matched::GroupedDataFrame, ::Union{Regex, AbstractString, Function, Signed, Symbol, Unsigned, Pair, Type, All, Between, Cols, InvertedIndex, AbstractVecOrMat}...; ungroup, renamecols, threads)
   @ DataFrames /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/DataFrames/LteEl/src/groupeddataframe/splitapplycombine.jl:958
  transform!(!Matched::DataFrame, ::Any...; renamecols, threads)
   @ DataFrames /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/DataFrames/LteEl/src/abstractdataframe/selection.jl:982


In [28]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Define the columns you want to scale
columns_to_scale = names(X_train)

# Create copies of the original data frames
X_train_scaled = copy(X_train)
X_test_scaled = copy(X_test)

# Extract the data to be scaled as matrices
X_train_to_scale = Matrix(X_train_scaled[:, columns_to_scale])
X_test_to_scale = Matrix(X_test_scaled[:, columns_to_scale])

# Fit and transform the scaler on the training data
fit_transform!(scaler, X_train_to_scale)

# Transform the test data using the same scaler
transform!(scaler, X_test_to_scale)

# Replace the original values in the DataFrame with the scaled values
X_train_scaled[:, columns_to_scale] = X_train_to_scale
X_test_scaled[:, columns_to_scale] = X_test_to_scale

MethodError: MethodError: no method matching transform!(::PyCall.PyObject, ::Matrix{Float64})

Closest candidates are:
  transform!(!Matched::SubDataFrame, ::Any...; renamecols, threads)
   @ DataFrames /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/DataFrames/LteEl/src/abstractdataframe/selection.jl:985
  transform!(!Matched::GroupedDataFrame, ::Union{Regex, AbstractString, Function, Signed, Symbol, Unsigned, Pair, Type, All, Between, Cols, InvertedIndex, AbstractVecOrMat}...; ungroup, renamecols, threads)
   @ DataFrames /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/DataFrames/LteEl/src/groupeddataframe/splitapplycombine.jl:958
  transform!(!Matched::DataFrame, ::Any...; renamecols, threads)
   @ DataFrames /Users/iai/builds/e7x1Q22r/0/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/DataFrames/LteEl/src/abstractdataframe/selection.jl:982


In [29]:
# Standardize the data
known_behaviors[!, 1:4] = (known_behaviors[!, 1:4] .- mean(known_behaviors[!, 1:4], dims=1)) ./ std(known_behaviors[!, 1:4], dims=1)

UndefVarError: UndefVarError: `known_behaviors` not defined